# 🧠 GERAÇÃO DE EMBEDDINGS - Etapa 5/6

## 📋 O que este notebook faz

Este notebook **converte texto em vetores** usando inteligência artificial para busca semântica:

- 📚 **Carrega chunks** de `pipeline-data/chunks/chunks.jsonl`
- 🤖 **Usa modelo BAAI/bge-m3** (multilíngue, otimizado para português)
- ⚡ **Processa em lotes** de 32 chunks por vez para eficiência
- 🎯 **Normaliza vetores** para busca por similaridade cosseno
- 💾 **Salva embeddings** enriquecidos em JSONL

## 🔧 Modelo BAAI/bge-m3

- **1024 dimensões** por vetor (alta expressividade semântica)
- **Multilíngue** com foco em português e inglês
- **Normalização L2** para comparação eficiente
- **State-of-the-art** para busca semântica

## 📊 Output esperado

322 chunks com embeddings (1.3MB) salvos em `pipeline-data/embeddings/embeddings.jsonl`

---

## 🔧 Configuração e Preparação

In [1]:
import os
import json
from pathlib import Path
from sentence_transformers import SentenceTransformer
import numpy as np
import time
from datetime import datetime

# Marcar início da execução
stage_start = time.time()
start_timestamp = datetime.now().isoformat() + "Z"

# Configuração
MODEL_NAME = os.getenv("EMBEDDING_MODEL", "BAAI/bge-m3")
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "32"))

# Diretórios
chunks_dir = Path("pipeline-data/chunks")
embeddings_dir = Path("pipeline-data/embeddings")
embeddings_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório embeddings
for f in embeddings_dir.glob("*"):
    if f.is_file():
        f.unlink()

print(f"Modelo: {MODEL_NAME}")
print(f"Batch size: {BATCH_SIZE}")

Modelo: BAAI/bge-m3
Batch size: 32


## 🤖 Carregamento do Modelo

In [2]:
# Carregar modelo
print("Carregando modelo de embeddings...")
model = SentenceTransformer(MODEL_NAME)
embedding_dim = model.get_sentence_embedding_dimension()

print(f"✅ Modelo carregado: {MODEL_NAME}")
print(f"Dimensões: {embedding_dim}")

Carregando modelo de embeddings...


✅ Modelo carregado: BAAI/bge-m3
Dimensões: 1024


In [3]:
# Carregar chunks
chunks_file = chunks_dir / "chunks.jsonl"

if not chunks_file.exists():
    raise FileNotFoundError(f"Arquivo de chunks não encontrado: {chunks_file}")

chunks = []
with open(chunks_file, "r", encoding="utf-8") as f:
    for line in f:
        chunk = json.loads(line)
        chunks.append(chunk)

print(f"Chunks carregados: {len(chunks)}")

# Mostrar alguns exemplos
for i, chunk in enumerate(chunks[:3]):
    preview = chunk["text"][:50] + "..." if len(chunk["text"]) > 50 else chunk["text"]
    print(f"  {i+1}. {preview}")

Chunks carregados: 322
  1. # Visão Geral do Self Checkout

## Introdução

Est...
  2. O objetivo desta documentação é descrever o fluxo ...
  3. - Processo de identificação do cliente via CPF ou ...


## 📚 Processamento de Chunks

In [4]:
# Gerar embeddings
print("Gerando embeddings...")

# Extrair textos dos chunks
texts = [chunk["text"] for chunk in chunks]

# Gerar embeddings em lotes
embeddings = model.encode(
    texts,
    batch_size=BATCH_SIZE,
    show_progress_bar=True,
    normalize_embeddings=True
)

print(f"✅ Embeddings gerados: {len(embeddings)}")
print(f"Formato: {embeddings.shape}")

Gerando embeddings...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

✅ Embeddings gerados: 322
Formato: (322, 1024)


In [5]:
# Combinar chunks com embeddings
chunks_with_embeddings = []

for chunk, embedding in zip(chunks, embeddings):
    chunk_with_embedding = {
        "chunk_id": chunk["chunk_id"],
        "source_document": chunk["source_document"],
        "chunk_index": chunk["chunk_index"],
        "text": chunk["text"],
        "char_count": chunk["char_count"],
        "embedding": embedding.tolist(),
        "embedding_model": MODEL_NAME,
        "embedding_dimensions": len(embedding)
    }
    chunks_with_embeddings.append(chunk_with_embedding)

print(f"Chunks com embeddings: {len(chunks_with_embeddings)}")

Chunks com embeddings: 322


## 💾 Armazenamento e Estatísticas

In [6]:
# Salvar embeddings
embeddings_file = embeddings_dir / "embeddings.jsonl"

with open(embeddings_file, "w", encoding="utf-8") as f:
    for chunk_data in chunks_with_embeddings:
        f.write(json.dumps(chunk_data, ensure_ascii=False) + "\n")

print(f"✅ Embeddings salvos: {embeddings_file}")

# Estatísticas
total_size_mb = (len(chunks_with_embeddings) * embedding_dim * 4) / (1024 * 1024)  # float32
avg_magnitude = np.mean([np.linalg.norm(chunk["embedding"]) for chunk in chunks_with_embeddings])

print(f"\n📊 Estatísticas:")
print(f"  Total embeddings: {len(chunks_with_embeddings)}")
print(f"  Dimensões: {embedding_dim}")
print(f"  Tamanho total: {total_size_mb:.1f} MB")
print(f"  Magnitude média: {avg_magnitude:.3f}")
print(f"  Modelo: {MODEL_NAME}")

✅ Embeddings salvos: pipeline-data/embeddings/embeddings.jsonl

📊 Estatísticas:
  Total embeddings: 322
  Dimensões: 1024
  Tamanho total: 1.3 MB
  Magnitude média: 1.000
  Modelo: BAAI/bge-m3


## 📊 Relatório de Execução

In [7]:
# Calcular duração
stage_duration = time.time() - stage_start

# Carregar relatório existente
report_path = Path("pipeline-data/report.json")
if report_path.exists():
    with open(report_path, "r") as f:
        report = json.load(f)
else:
    report = {"stages": [], "context": {}, "summary": {}}

# Atualizar contexto com modelo de embedding
report["context"]["embedding_model"] = MODEL_NAME

# Adicionar informações desta etapa
stage_report = {
    "stage": 5,
    "name": "Geração de Embeddings",
    "status": "SUCCESS" if len(chunks_with_embeddings) > 0 else "FAILED",
    "start_time": start_timestamp,
    "duration_seconds": round(stage_duration, 2),
    "results": {
        "chunks_loaded": len(chunks),
        "embeddings_generated": len(embeddings),
        "embedding_dimensions": embedding_dim,
        "batch_size": BATCH_SIZE,
        "storage_size_mb": round(total_size_mb, 2),
        "avg_magnitude": round(avg_magnitude, 3)
    }
}

# Adicionar ou atualizar stage no relatório
stages_updated = False
for i, stage in enumerate(report["stages"]):
    if stage["stage"] == 5:
        report["stages"][i] = stage_report
        stages_updated = True
        break

if not stages_updated:
    report["stages"].append(stage_report)

# Atualizar timestamp
report["summary"]["last_update"] = datetime.now().isoformat() + "Z"

# Salvar relatório atualizado
with open(report_path, "w") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\n📊 Relatório atualizado: {report_path}")
print(f"⏱️ Duração da etapa: {stage_duration:.2f}s")


📊 Relatório atualizado: pipeline-data/report.json
⏱️ Duração da etapa: 68.54s
